<a href="https://colab.research.google.com/github/kuds/rl-car-racing/blob/main/%5BCar%20Racing%5D%20Proximal%20Policy%20Optimization%20(PPO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install swig moviepy

In [2]:
!pip install gymnasium gymnasium[box2d] stable_baselines3

In [3]:
import gymnasium
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.vec_env import VecTransposeImage
from stable_baselines3.common.evaluation import evaluate_policy

import torch
import time
torch.backends.cudnn.benchmark = True

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Create CarRacing environment

#env = gymnasium.make("CarRacing-v2", render_mode="rgb_array")
env = make_vec_env("CarRacing-v2", n_envs=1)
env = VecTransposeImage(env)

eval_callback = EvalCallback(env, best_model_save_path="./logs/", log_path="./logs/", eval_freq=5000, deterministic=True, render=False)

# Initialize PPO
model = PPO('CnnPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=300000,  progress_bar=True, callback=eval_callback)

# Save the model
model.save("ppo_car_racing")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Using cuda device


Output()

Eval num_timesteps=500, episode_reward=-90.09 +/- 1.12

Episode length: 1000.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 1e+03    |
|    mean_reward     | -90.1    |
| time/              |          |
|    total_timesteps | 500      |
---------------------------------


...

---------------------------------
| eval/              |          |
|    mean_ep_length  | 287      |
|    mean_reward     | -89.5    |
| time/              |          |
|    total_timesteps | 4000     |
---------------------------------
-----------------------------
| time/              |      |
|    fps             | 8    |
|    iterations      | 2    |
|    time_elapsed    | 482  |
|    total_timesteps | 4096 |
-----------------------------


Eval num_timesteps=4500, episode_reward=-32.87 +/- 14.83

Episode length: 1000.00 +/- 0.00

-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1e+03       |
|    mean_reward          | -32.9       |
| time/                   |             |
|    total_timesteps      | 4500        |
| train/                  |             |
|    approx_kl            | 0.006941966 |
|    clip_fraction        | 0.083       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.26       |
|    explained_variance   | 0.332       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.361       |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.00752    |
|    std                  | 1           |
|    value_loss           | 0.893       |
-----------------------------------------


New best mean reward!